In [24]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Read csv files

In [187]:
file_train = '../data/wikipedia_train.csv'
file_test = '../data/wikipedia_test.csv'

df_train = pd.read_csv(file_train)
df_test = pd.read_csv(file_test)

In [46]:
def change_df(df):
    columns = df.columns.values[1:]
    df = pd.melt(df, id_vars='Page', value_vars=columns, value_name='Visits', var_name='Date')
    df['Date'] = pd.to_datetime(df['Date'])
    df['Dayweek'] = df['Date'].dt.dayofweek
    # df = df.set_index('Page')
    return df

In [188]:
df_train = change_df(df_train)
df_test = change_df(df_test)

In [49]:
df_train.head()

,Page,Date,Visits,Dayweek
0,15._November_de.wikipedia.org_desktop_all-agents,2015-07-01,32.0,2
1,2012_(film)_fr.wikipedia.org_all-access_spider,2015-07-01,2.0,2
2,2016_FIFA_U-20女子ワールドカップ_ja.wikipedia.org_all-a...,2015-07-01,1.0,2
3,2016_UEFA_Europa_League_Final_en.wikipedia.org...,2015-07-01,3.0,2
4,2016_in_video_gaming_en.wikipedia.org_all-acce...,2015-07-01,24.0,2


In [175]:
def get_medians(df, days):
    date_window = len(df['Page'].unique()) * 7 * days
    grouped = df.iloc[-date_window:].groupby(['Page','Dayweek'])['Visits'].median().fillna(0)
    return grouped

def form_series(grouped, test_data):
    array = []
    for item in test_data.values:
        array.append(grouped.loc[item[0], item[3]])
    return array

def pandas_smape(df):
    df.fillna(0, inplace=True)
    df["SMAPE"] = 200 * np.abs(df["Visits"] - df["pred_Visits"]) / (df["Visits"] + df["pred_Visits"])
    df["SMAPE"].fillna(0, inplace=True)
    return np.mean(df["SMAPE"])

In [193]:
smapes = dict()
rang = range(1, len(df_train) // len(df_train['Page'].unique()) // 7)
for i in rang:
    grouped = get_medians(df_train, i)
    df_test['pred_Visits'] = form_series(grouped, df_test)
    smapes[i] = pandas_smape(df_test)
    
minimum= sorted(list(smapes.items()), key=lambda x: x[1])[0]
print('Max accuracy by: {}: {}'.format(minimum[0], minimum[1]))

Max accuracy by: 27: 49.83547039727196
(27, 49.83547039727196)


In [198]:
len(df_train) // len(df_train['Page'].unique()) // 7

61